In [1]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')

from sklearn.feature_selection.univariate_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import random
import itertools

import sys
import os
import argparse
from sklearn.pipeline import Pipeline
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer
import six
from abc import ABCMeta
from scipy import sparse
from scipy.sparse import issparse
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils import check_X_y, check_array
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.preprocessing import normalize, binarize, LabelBinarizer
from sklearn.svm import LinearSVC

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict
from keras.layers.convolutional import Convolution1D
from keras import backend as K

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
plt.style.use('ggplot')

Using TensorFlow backend.


In [2]:
def review_to_wordlist(review, remove_stopwords = True):
    
    reveiw_text = BeautifulSoup(review).get_text()
    #beautifulsoup :http이런거 없애줌 
    
    review_text = re.sub('[^a-zA-Z]'," ", review)
    #review_text에 영어만 넣기 
    
    words = review_text.lower().split()
    #소문자로 바꿔주고 그것들을 분리해준다 
    
    if remove_stopwords:
        stops = set(stopwords.words('english'))
        words = [w for w in words if not w in stops]
        #stops에 영어의 불용어를 넣어줌 
        #words는 소문자로 변환되고 띄어져있는 것이며 
        #stops에 있는 불용어를 제외하고 넣어줌 
        
    b = []
    stemmer = english_stemmer
    for word in words:
        b.append(stemmer.stem(word))
        #words에 전처리된것들의 어간들만 추출 
        
    return(b)

In [17]:
data = pd.read_csv('./Amazon_Unlocked_Mobile.csv')

In [18]:
data

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0
...,...,...,...,...,...,...
413835,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,another great deal great price,0.0
413836,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,Ok,0.0
413837,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,Passes every drop test onto porcelain tile!,0.0
413838,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,I returned it because it did not meet my needs...,0.0


In [19]:
data.isnull().sum()

Product Name        0
Brand Name      65171
Price            5933
Rating              0
Reviews            62
Review Votes    12296
dtype: int64

In [21]:
data = data.dropna()

In [23]:
data.isnull().sum()

Product Name    0
Brand Name      0
Price           0
Rating          0
Reviews         0
Review Votes    0
dtype: int64

In [22]:
train, test = train_test_split(data, test_size = 0.3)
#학습과 테스트에 사용하는 데이터의 비율은 7:3

In [24]:
clean_train_reviews = []
for review in train['Reviews']:
    clean_train_reviews.append( " ".join(review_to_wordlist(review)))
    
clean_test_reviews = []
for review in test['Reviews']:
    clean_test_reviews.append( " ".join(review_to_wordlist(review)))

/Users/Moon/opt/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.amazon.com/gp/product/B013YDFH3Y?redirect=true&ref_=cm_cr_ryp_prd_ttl_sol_0" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/Moon/opt/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:282: UserWarning: "https://www.amazon.com/gp/product/B00U8KSNB0/ref=cm_cr_ryp_prd_ttl_sol_22" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/Moon/opt/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:282: UserWarning: "https://www.amazon.com/dp/B00K15KRV6/ref=cm_cr_ryp_prd_ttl_sol_22" looks like a URL. B

In [25]:
clean_train_reviews
#전처리가 된것을 확인 할 수 있다.

['love',
 'item ok present sent one west indi said thing assum ok work well',
 'nice look difficult use use time use old sim card',
 'batteri die super fast',
 'phone fault applic nexus show scratch back realli like',
 'got phone yesterday night turn morn saw charg hold minut reasoni want replac option garbag phone dollar want money back',
 'perfect recommend',
 'arriv perfect condit reset factori great shape work describ',
 'bought use use cheap borescop work fine',
 'okay batteri life hour',
 'item min turn stope work screen goe sleep u never get back batteri kick within day phone stope charg worst item ever buy amazon dont recomend',
 'love light',
 'phone dosen work sim card',
 'good',
 'good quailti',
 'excel',
 'quick ship exact expect great price',
 'bought phone advertis unlock intern phone unlock advertis addit cost unlock',
 'blackberri fan phone everyth need fear would pri cold dead hand',
 'love best featur fingerprint sensor pure android experi',
 'ship rather quick though

In [26]:
len(clean_train_reviews)

234034

### TF-idf 변환 

In [16]:
vectorizer = TfidfVectorizer( min_df = 2, max_df = 0.95, 
                             max_features = 200000, 
                             ngram_range = ( 1, 4),  
                             sublinear_tf = True)
vectorizer = vectorizer.fit(clean_train_reviews)
train_features = vectorizer.transform(clean_train_reviews)

test_features = vectorizer.transform(clean_test_reviews)
#min_df : 한 문서에서 최소 2번이상 단어나 와야함
#max_df : 전체 문서에서 95%이상 나타나는 단어들은 무시 
#max_features : 각 단어들이 df에 열이 되는데 열을 20000개까지만 설정(늘리고 싶으면 늘려도 됨 대신 계산량 증가 )
#ngram_range : 몇개의 단어를 묶을건지 지정 위에서는 최소 1개부터 최대 4개까지의 단어묶음 
#sublinear_tf : tf값이 무한정 커지는것을 막아주는 역활 

In [17]:
train_features

<14583x74695 sparse matrix of type '<class 'numpy.float64'>'
	with 480813 stored elements in Compressed Sparse Row format>

In [21]:
fselect = SelectKBest(chi2 , k=10000) #10000개의 좋은 변수들만 추출 

train_features = fselect.fit_transform(train_features, train["Rating"])

test_features = fselect.transform(test_features)
#SelectKBest : 성능이 좋은 변수만 사용하여 처리해주는 것 

In [41]:
train_features
#약 76000개의 행에서 10000개의 행으로 줄어들었다 

<14585x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 208292 stored elements in Compressed Sparse Row format>

In [22]:
model1 = MultinomialNB(alpha=0.001)
model1.fit( train_features, train["Rating"] )
#다항분포 나이브베이즈모델 

model2 = SGDClassifier(loss='modified_huber',random_state=0, shuffle=True)
model2.fit( train_features, train["Rating"] )
#modified_huber를 사용한 이유는 이상치에 많은 영향을 안받는다고 생각하여 넣은것 같다
#이게 맞는지 모르겠음 다음번에 물어봐야겠다 

model3 = RandomForestClassifier()
model3.fit( train_features, train["Rating"] )
#랜덤포레스트 

model4 = GradientBoostingClassifier()
model4.fit( train_features, train["Rating"] )
#부스팅 계열의 알고리즘 
#부스팅 : 예측을 반복하면서 잔차들이 생기는데 이 잔차들을 줄여나가면서 모형을 만듬 

pred_1 = model1.predict( test_features.toarray() )
pred_2 = model2.predict( test_features.toarray() )
pred_3 = model3.predict( test_features.toarray() )
pred_4 = model4.predict( test_features.toarray() )


/Users/Moon/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


# ======================================= 04.24

In [23]:
class NBSVM(six.with_metaclass(ABCMeta, BaseEstimator, ClassifierMixin)):

    def __init__(self, alpha=1.0, C=1.0, max_iter=10000):
        self.alpha = alpha
        self.max_iter = max_iter
        self.C = C
        self.svm_ = [] # fuggly

    def fit(self, X, y):
        X, y = check_X_y(X, y, 'csr')
        _, n_features = X.shape

        labelbin = LabelBinarizer()
        Y = labelbin.fit_transform(y)
        self.classes_ = labelbin.classes_
        if Y.shape[1] == 1:
            Y = np.concatenate((1 - Y, Y), axis=1)

        # LabelBinarizer().fit_transform() returns arrays with dtype=np.int64.
        # so we don't have to cast X to floating point
        Y = Y.astype(np.float64)

        # Count raw events from data
        n_effective_classes = Y.shape[1]
        self.class_count_ = np.zeros(n_effective_classes, dtype=np.float64)
        self.ratios_ = np.full((n_effective_classes, n_features), self.alpha,
                                 dtype=np.float64)
        self._compute_ratios(X, Y)

        # flugglyness
        for i in range(n_effective_classes):
            X_i = X.multiply(self.ratios_[i])
            svm = LinearSVC(C=self.C, max_iter=self.max_iter)
            Y_i = Y[:,i]
            svm.fit(X_i, Y_i)
            self.svm_.append(svm) 

        return self

    def predict(self, X):
        n_effective_classes = self.class_count_.shape[0]
        n_examples = X.shape[0]

        D = np.zeros((n_effective_classes, n_examples))

        for i in range(n_effective_classes):
            X_i = X.multiply(self.ratios_[i])
            D[i] = self.svm_[i].decision_function(X_i)
        
        return self.classes_[np.argmax(D, axis=0)]
        
    def _compute_ratios(self, X, Y):
        """Count feature occurrences and compute ratios."""
        if np.any((X.data if issparse(X) else X) < 0):
            raise ValueError("Input X must be non-negative")

        self.ratios_ += safe_sparse_dot(Y.T, X)  # ratio + feature_occurrance_c
        normalize(self.ratios_, norm='l1', axis=1, copy=False)
        row_calc = lambda r: np.log(np.divide(r, (1 - r)))
        self.ratios_ = np.apply_along_axis(row_calc, axis=1, arr=self.ratios_)
        check_array(self.ratios_)
        self.ratios_ = sparse.csr_matrix(self.ratios_)

        #p_c /= np.linalg.norm(p_c, ord=1)
        #ratios[c] = np.log(p_c / (1 - p_c))


def f1_class(pred, truth, class_val):
    n = len(truth)

    truth_class = 0
    pred_class = 0
    tp = 0

    for ii in range(0, n):
        if truth[ii] == class_val:
            truth_class += 1
            if truth[ii] == pred[ii]:
                tp += 1
                pred_class += 1
                continue;
        if pred[ii] == class_val:
            pred_class += 1

    precision = tp / float(pred_class)
    recall = tp / float(truth_class)

    return (2.0 * precision * recall) / (precision + recall)


def semeval_senti_f1(pred, truth, pos=2, neg=0): 

    f1_pos = f1_class(pred, truth, pos)
    f1_neg = f1_class(pred, truth, neg)

    return (f1_pos + f1_neg) / 2.0;


def main(train_file, test_file, ngram=(1, 3)):
    print('loading...')
    train = pd.read_csv(train_file, delimiter='\t', encoding='utf-8', header=0,
                        names=['text', 'label'])

    # to shuffle:
    #train.iloc[np.random.permutation(len(df))]

    test = pd.read_csv(test_file, delimiter='\t', encoding='utf-8', header=0,
                        names=['text', 'label'])

    print('vectorizing...')
    vect = CountVectorizer()
    classifier = NBSVM()

    # create pipeline
    clf = Pipeline([('vect', vect), ('nbsvm', classifier)])
    params = {
        'vect__token_pattern': r"\S+",
        'vect__ngram_range': ngram, 
        'vect__binary': True
    }
    clf.set_params(**params)

    #X_train = vect.fit_transform(train['text'])
    #X_test = vect.transform(test['text'])

    print('fitting...')
    clf.fit(train['text'], train['label'])

    print('classifying...')
    pred = clf.predict(test['text'])
   
    print('testing...')
    acc = accuracy_score(test['label'], pred)
    f1 = semeval_senti_f1(pred, test['label'])
    print('NBSVM: acc=%f, f1=%f' % (acc, f1))

In [24]:
model5 = NBSVM(C=0.01)
model5.fit( train_features, train["Rating"] )

pred_5 = model5.predict( test_features )

In [25]:
print(classification_report(test['Rating'], pred_2, target_names=['1','2','3','4','5']))

              precision    recall  f1-score   support

           1       0.67      0.79      0.72      1084
           2       0.47      0.12      0.19       371
           3       0.40      0.11      0.17       514
           4       0.34      0.17      0.23       877
           5       0.74      0.93      0.82      3405

    accuracy                           0.68      6251
   macro avg       0.52      0.42      0.43      6251
weighted avg       0.63      0.68      0.63      6251



In [26]:
print('prediction 1 accuracy: ', accuracy_score(test['Rating'], pred_1))
print('prediction 2 accuracy: ', accuracy_score(test['Rating'], pred_2))
print('prediction 3 accuracy: ', accuracy_score(test['Rating'], pred_3))
print('prediction 4 accuracy: ', accuracy_score(test['Rating'], pred_4))
print('prediction 5 accuracy: ', accuracy_score(test['Rating'], pred_5))

prediction 1 accuracy:  0.6654935210366342
prediction 2 accuracy:  0.6838905775075987
prediction 3 accuracy:  0.6616541353383458
prediction 4 accuracy:  0.6478963365861462
prediction 5 accuracy:  0.6866101423772196


### MLP(Multilayer Perceptrin)

In [65]:
batch_size = 32 
nb_classes = 5
EPOCHS = 5

In [35]:
vectorizer = TfidfVectorizer( min_df=2, max_df=0.95, 
                              max_features = 20000, 
                              ngram_range = ( 1, 3 ),
                              sublinear_tf = True )

vectorizer = vectorizer.fit(clean_train_reviews)
train_features = vectorizer.transform(clean_train_reviews)

test_features = vectorizer.transform(clean_test_reviews)

In [37]:
type(train_features)

scipy.sparse.csr.csr_matrix

In [46]:
from scipy.sparse import dok_matrix
m = dok_matrix([[1, 0],
                [0, 1]])
print(m)

  (0, 0)	1
  (1, 1)	1


In [42]:
type(m)

scipy.sparse.dok.dok_matrix

In [45]:
c=m.toarray()
print(c)

[[1 0]
 [0 1]]


In [47]:
type(c)
#toarray는 matrix에서 array로 바꿔주는것같은데 
#m과 c는 차이가 난다 왜지?

numpy.ndarray

In [70]:
X_train = train_features.toarray()
X_test = test_features.toarray()

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
Y_train = np.array(train['Rating']-1)
Y_test = np.array(test['Rating']-1)

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


# pre-processing: divide by max and substract mean
scale = np.max(X_train)
X_train /= scale
X_test /= scale

mean = np.mean(X_train)
X_train -= mean
X_test -= mean

input_dim = X_train.shape[1]

# Here's a Deep Dumb MLP (DDMLP)
model = Sequential()
model.add(Dense(256, input_dim=input_dim))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# we'll use categorical xent for the loss, and RMSprop as the optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

print("Training...")
model.fit(X_train, Y_train, epochs = EPOCHS, batch_size=16, validation_split=0.1)

print("Generating test predictions...")
preds = model.predict_classes(X_test, verbose=0)

X_train shape: (14583, 20000)
X_test shape: (6251, 20000)
Training...
Train on 13124 samples, validate on 1459 samples
Epoch 1/5
13124/13124 [==============================] - 33s 3ms/step - loss: 0.9736 - accuracy: 0.6377 - val_loss: 0.8259 - val_accuracy: 0.6957
Epoch 2/5
13124/13124 [==============================] - 29s 2ms/step - loss: 0.6435 - accuracy: 0.7645 - val_loss: 0.8687 - val_accuracy: 0.6950
Epoch 3/5
13124/13124 [==============================] - 29s 2ms/step - loss: 0.3941 - accuracy: 0.8647 - val_loss: 1.0336 - val_accuracy: 0.6628
Epoch 4/5
13124/13124 [==============================] - 34s 3ms/step - loss: 0.2417 - accuracy: 0.9227 - val_loss: 1.2346 - val_accuracy: 0.6710
Epoch 5/5
13124/13124 [==============================] - 33s 3ms/step - loss: 0.1807 - accuracy: 0.9425 - val_loss: 1.4385 - val_accuracy: 0.6429
Generating test predictions...


### 오버피팅이 일어난듯하다
#### epoch를 5번만 했음에도 오버피팅이 난것은 이상하다
##### 원인파악해봐야할듯

In [52]:
print('prediction 6 accuracy: ', accuracy_score(test['Rating'], preds+1))

prediction 6 accuracy:  0.6846904495280756


In [4]:
import tensorflow as tf
print(tf.__version__)

1.15.0


In [69]:
import keras
print(keras.__version__)

2.3.1
